In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from matplotlib import cm
from PIL import Image
from tqdm import tqdm

from counterfactuals.datasets.blobs import BlobsDataset
from counterfactuals.cf_methods.ppcef import PPCEF
from counterfactuals.losses import MulticlassDiscLoss
from counterfactuals.discriminative_models import MultinomialLogisticRegression
from counterfactuals.generative_models import MaskedAutoregressiveFlow

In [3]:
def plot_model_distribution(model):
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(20, 12)

    xline = torch.linspace(-0.1, 1.2, 200)
    yline = torch.linspace(-0.1, 1.2, 200)
    xgrid, ygrid = torch.meshgrid(xline, yline)
    xyinput = torch.cat([xgrid.reshape(-1, 1), ygrid.reshape(-1, 1)], dim=1)

    with torch.no_grad():
        zgrid0 = model(xyinput, torch.zeros(40000, 1)).exp().reshape(200, 200)
        zgrid1 = model(xyinput, torch.ones(40000, 1)).exp().reshape(200, 200)
        zgrid2 = model(xyinput, 2*torch.ones(40000, 1)).exp().reshape(200, 200)

    zgrid0 = zgrid0.numpy()
    zgrid1 = zgrid1.numpy()
    zgrid2 = zgrid2.numpy()

    _ = ax.contour(
        xgrid.numpy(),
        ygrid.numpy(),
        zgrid0,
        levels=10,
        cmap="Greys",
        linewidths=0.4,
        antialiased=True,
    )
    _ = ax.contour(
        xgrid.numpy(),
        ygrid.numpy(),
        zgrid1,
        levels=10,
        cmap="Oranges",
        linewidths=0.4,
        antialiased=True,
    )
    _ = ax.contour(
        xgrid.numpy(),
        ygrid.numpy(),
        zgrid2,
        levels=10,
        cmap="Blues",
        linewidths=0.4,
        antialiased=True,
    )
    return ax

In [4]:
def add_metrics_textbox(metrics_series, ax):
    text_str = '\n'.join(f"{metric}: {value:.3f}" for metric, value in metrics_series.items())

    props = dict(boxstyle='round', facecolor='wheat', alpha=0.7)
    ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=10, 
            verticalalignment='top', bbox=props)
    return ax


def plot_all_cf(disc_model, flow, X_test, X_cf, metrics):
    assert X_test.shape[1] == X_cf.shape[1], "Sizes of test set and counterfactuals is not equal."

    ax = plot_model_distribution(flow)

    num_classes = list(disc_model.parameters())[0].shape[0]
    
    for i in range(num_classes):
        # Classifier Line
        w1, w2 = list(disc_model.parameters())[0][i].detach().cpu().numpy()
        b = list(disc_model.parameters())[1][i].detach().cpu().numpy().item()
        c = -b / w2
        m = -w1 / w2
        xmin, xmax = -0.1, 1.2
        ymin, ymax = -0.1, 1.2
        xd = np.array([xmin, xmax])
        yd = m * xd + c
        plt.plot(xd, yd, "red", lw=2.0, ls="dashed")
        # plt.axis("off")
    
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=matplotlib.colormaps["tab10"], s=50, alpha=0.8)
    ax.scatter(X_cf[:, 0], X_cf[:, 1], c="orange", s=50, alpha=0.8)
    
    for i in range(len(X_test)):
        ax.arrow(
            X_test[i, 0],
            X_test[i, 1],
            X_cf[i, 0] - X_test[i, 0],
            X_cf[i, 1] - X_test[i, 1],
            width=0.001,
            lw=0.001,
            length_includes_head=True,
            alpha=0.5,
            color="k",
        )

    ax = add_metrics_textbox(metrics, ax)
    
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.1, 1.1)
    ax.set_xticks([])
    ax.set_yticks([])
    
    return ax

In [5]:
dataset = BlobsDataset(file_path="../data/blobs.csv")
datasets = iter(dataset.get_cv_splits())

for i in range(5):
    X_train, X_test, y_train, y_test = next(datasets)  # yields splits one by one
    
    disc_model = MultinomialLogisticRegression(dataset.X_train.shape[1], len(np.unique(dataset.y)))
    disc_model.load(f"../models/BlobsDataset/disc_model_{i}_MultinomialLogisticRegression.pt")
    
    flow = MaskedAutoregressiveFlow(
        dataset.X_train.shape[1],
        hidden_features=4,
        num_blocks_per_layer=2,
        num_layers=5,
        context_features=1,
    )
    flow.load(f"../models/BlobsDataset/gen_model_{i}_MaskedAutoregressiveFlow.pt")
    
    cf = PPCEF(
        gen_model=flow,
        disc_model=disc_model,
        disc_model_criterion=MulticlassDiscLoss(),
        neptune_run=None,
    )
    
    median_prob = (
        flow.predict_log_prob(dataset.train_dataloader(batch_size=64, shuffle=False))
        .median()
        .item()
    )

    for method in ['artelth20', 'cbce', 'CEM', 'ppcef', 'wach']: # 'CEGP'
        X_cf = pd.read_csv(
            f"../models/BlobsDataset/{method}/counterfactuals_MultinomialLogisticRegression_{i}.csv"
        ).values
        
        metrics = pd.read_csv(
            f"../models/BlobsDataset/{method}/metrics_MultinomialLogisticRegression_cv.csv"
        )
        
        print(i, method)
        print(X_cf.shape, X_test.shape)
    
        ax = plot_all_cf(disc_model, flow, X_test, X_cf, metrics.iloc[i])
        ax.set_title(method)
        plt.tight_layout()
        plt.savefig(f"../models/BlobsDataset/{method}/visualization_{i}.png")
        plt.close()


0 artelth20
(300, 2) (300, 2)


/opt/homebrew/anaconda3/envs/counterfactuals/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


0 cbce
(300, 2) (300, 2)
0 CEM
(300, 2) (300, 2)
0 ppcef
(300, 2) (300, 2)
0 wach
(300, 2) (300, 2)
1 artelth20
(300, 2) (300, 2)
1 cbce
(300, 2) (300, 2)
1 CEM
(300, 2) (300, 2)
1 ppcef
(300, 2) (300, 2)
1 wach
(300, 2) (300, 2)
2 artelth20
(300, 2) (300, 2)
2 cbce
(300, 2) (300, 2)
2 CEM
(300, 2) (300, 2)
2 ppcef
(300, 2) (300, 2)
2 wach
(300, 2) (300, 2)
3 artelth20
(300, 2) (300, 2)
3 cbce
(300, 2) (300, 2)
3 CEM
(300, 2) (300, 2)
3 ppcef
(300, 2) (300, 2)
3 wach
(300, 2) (300, 2)
4 artelth20
(299, 2) (299, 2)
4 cbce
(299, 2) (299, 2)
4 CEM
(299, 2) (299, 2)
4 ppcef
(299, 2) (299, 2)
4 wach
(299, 2) (299, 2)


In [6]:
def create_grid_image(folders, output_filename):
    """Creates a grid image combining images from multiple folders, with automatic grid calculation.

    Args:
        folders (list): A list of paths to folders containing images.
        output_filename (str): Name of the output image file.
    """

    images = []
    for folder in folders:
        for filename in os.listdir(folder):
            if filename.endswith((".png", ".jpg")):
                images.append(Image.open(os.path.join(folder, filename)))

    # Calculate grid dimensions
    num_images = len(images)
    rows = math.ceil(math.sqrt(num_images))
    cols = math.ceil(num_images / rows)

    # Calculate image dimensions (assuming all have reasonably similar size)
    image_width, image_height = images[0].size

    grid_width = image_width * cols
    grid_height = image_height * rows
    grid_image = Image.new('RGB', (grid_width, grid_height))

    index = 0
    for row in range(rows):
        for col in range(cols):
            if index < num_images:  # Prevent going out of bounds
                grid_image.paste(images[index], (col * image_width, row * image_height))
            index += 1

    grid_image.save(output_filename)


folders = [f"../models/BlobsDataset/{method}" for method in ['artelth20', 'cbce', 'CEM', 'ppcef', 'wach']] # 'CEGP',
output_filename = f"../models/BlobsDataset/counterfactuals_logistic_regression_comparison.png"

create_grid_image(folders, output_filename)
